The purpose of this code is to calculate the circularity index.

Import Libraries

In [7]:
import os
import numpy as np
import pandas as pd
import skimage.io as io
from skimage.filters import gaussian
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import czifile as czi

Define Sub Functions

In [8]:
def load_images(folder_path):
    """Load all image filenames from the specified folder."""
    return [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

def preprocess_image(image_path):
    """Preprocess the image by reading, squeezing, and extracting the first frame."""
    image = czi.imread(image_path)
    return np.squeeze(image)[0, :, :]

def apply_gaussian_filter(image, sigma=2):
    """Apply Gaussian filtering to the image."""
    return gaussian(image, sigma=sigma)

def threshold_image(image, threshold_value=0.15):
    """Threshold the image based on the specified threshold value."""
    return image > threshold_value

def label_inclusions(binary_image):
    """Label the inclusions in the binary image using connected component analysis."""
    return label(binary_image)

def calculate_circularity(area, perimeter):
    """Calculate the circularity index of an inclusion."""
    if perimeter == 0:
        return "perimeter=0"
    return (4 * np.pi * area) / (perimeter ** 2)

def analyze_inclusions(props, size_threshold=57, circularity_warning_threshold=1.1):
    """
    Analyze inclusions based on their size and circularity.

    Explanation:
    - Inclusions are categorized into two groups: those up to size 57 inclusive, and those greater than 57.
    - Inclusions with a size of 57 or less often have a circularity index above 1, indicating difficulty in
      accurately calculating their circularity.
    - All inclusions with a size of 57 or less are excluded to eliminate artefactual measurements, even if some
      have a circularity index below 1.
    - Results are rounded to one decimal place, and warnings are added for inclusions with a circularity index >= 1.1.
    """
    results_greater = {
        'images': [], 'labels': [], 'circularity_indices': [], 
        'sizes': [], 'perimeters': [], 'warnings': []
    }
    results_less = {
        'images': [], 'labels': [], 'circularity_indices': [], 
        'sizes': [], 'perimeters': []
    }

    for prop in props:
        area = prop.area
        perimeter = prop.perimeter
        circularity_index = calculate_circularity(area, perimeter)

        if area > size_threshold:
            results_greater['sizes'].append(area)
            results_greater['perimeters'].append(perimeter)
            results_greater['labels'].append(prop.label)
            results_greater['circularity_indices'].append(circularity_index)
            warning = "circularity_index>=1.1" if perimeter > 0 and circularity_index >= circularity_warning_threshold else ""
            results_greater['warnings'].append(warning)
        else:
            results_less['sizes'].append(area)
            results_less['perimeters'].append(perimeter)
            results_less['labels'].append(prop.label)
            results_less['circularity_indices'].append(circularity_index)
    
    return results_greater, results_less

def save_to_excel(images, labels, circularity_indices, sizes, perimeters, warnings, file_name):
    """Save the analyzed data to an Excel file."""
    data = {
        'image': images,
        'inclusion_index': labels,
        'circularity_index': circularity_indices,
        'size': sizes,
        'perimeter': perimeters,
        'all_warnings': warnings
    }
    df = pd.DataFrame(data)
    df.to_excel(file_name, index=False)

Define Main Function

In [9]:
def main(folder_path):
    # Load all image files from the specified folder
    image_files = load_images(folder_path)
    
    # Initialize lists to store results for inclusions greater than 57
    images_greater_than_57, labels_greater_than_57, circularity_indices_greater_than_57, sizes_greater_than_57, perimeters_greater_than_57, warnings_greater_than_57 = [], [], [], [], [], []
    
    # Initialize lists to store results for inclusions less than or equal to 57
    images_less_than_57, labels_less_than_57, circularity_indices_less_than_57, sizes_less_than_57, perimeters_less_than_57 = [], [], [], [], []

    # Process each image file
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        
        # Preprocess the image: load, squeeze, and extract the first frame
        first_frame = preprocess_image(image_path)
        
        # Apply Gaussian filtering to smooth the image
        filtered_image = apply_gaussian_filter(first_frame)
        
        # Threshold the image to create a binary image
        thresholded_image = threshold_image(filtered_image)
        
        # Label the inclusions in the binary image
        labeled_inclusions = label_inclusions(thresholded_image)
        
        # Extract properties of the labeled regions (inclusions)
        props = regionprops(labeled_inclusions)
        
        # Analyze the inclusions based on size and circularity, categorizing them
        # into those greater than 57 and those less than or equal to 57
        results_greater, results_less = analyze_inclusions(props)

        # Store the results for inclusions greater than 57
        images_greater_than_57.extend([image_file] * len(results_greater['sizes']))
        labels_greater_than_57.extend(results_greater['labels'])
        circularity_indices_greater_than_57.extend(results_greater['circularity_indices'])
        sizes_greater_than_57.extend(results_greater['sizes'])
        perimeters_greater_than_57.extend(results_greater['perimeters'])
        warnings_greater_than_57.extend(results_greater['warnings'])

        # Store the results for inclusions less than or equal to 57
        images_less_than_57.extend([image_file] * len(results_less['sizes']))
        labels_less_than_57.extend(results_less['labels'])
        circularity_indices_less_than_57.extend(results_less['circularity_indices'])
        sizes_less_than_57.extend(results_less['sizes'])
        perimeters_less_than_57.extend(results_less['perimeters'])

    # Save the results for inclusions greater than 57 to an Excel file
    save_to_excel(images_greater_than_57, labels_greater_than_57, circularity_indices_greater_than_57, sizes_greater_than_57, perimeters_greater_than_57, warnings_greater_than_57, "circularity_index_output_greater_than_57.xlsx")
    
    # Save the results for inclusions less than or equal to 57 to another Excel file
    save_to_excel(images_less_than_57, labels_less_than_57, circularity_indices_less_than_57, sizes_less_than_57, perimeters_less_than_57, [""] * len(images_less_than_57), "circularity_index_output_less_than_equal_57.xlsx")


if __name__ == "__main__":
    main("test_images")